In [103]:
# python -m spacy download en
import spacy 
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import fitz
import re 

spa = spacy.load('en_core_web_sm')


In [62]:
doc = fitz.open('./books/1 A Game of Thrones.pdf')
text = ''
# for pag in doc:
#     text += pag.get_text()    

In [63]:
len(text)

1614141

In [67]:
#spacy doc
spacy_doc =spa(text[:10000])

In [68]:
i = 0
for d in spacy_doc.sents :
    print(d)
    # break
    i+=1
    if i ==20:
        break


Version History: 
 
2.0 - Reedited 4/25/10 by maelstrom385 
1.0 - Scanned 3/5/02 by sliph 
 
*missing appendix section 
 
 
 
Book One: A Song of Ice and Fire 
George R.R.
Martin 
 
 
 
 
 
PROLOGUE 
 
“We should start back,”, Gared urged as the woods began to grow dark around them.
“The 
wildlings are dead.”, 
  “Do the dead frighten you?”, Ser Waymar Royce asked with just the hint of a smile.

 Gared did not rise to the bait.
He was an old man, past fifty, and he had seen the lordlings come 
and go.
“Dead is dead,”, he said.
“We have no business with the dead.”, 
  “Are they dead?”, Royce asked softly.
“What proof have we?”, 
  “Will saw them,”, Gared said.
“If he says they are dead, that’s proof enough for me.”, 
Will had known they would drag him into the quarrel sooner or later.
He wished it had been later 
rather than sooner.
“My mother told me that dead men sing no songs,”, he put in. 
  
“My wet nurse said the same thing, Will,”, Royce replied.
“Never believe anything you hear 

In [84]:
sentances = []
entities = []
for sent in spacy_doc.sents :
    sentances.append(sent)
    entities.append([entity.text for entity in sent.ents  if entity.label_ not in['CARDINAL','DATE','TIME','QUANTITY','WORK_OF_ART']])


In [107]:
sentances = []
text= ""
for pag in doc:
    pag = re.sub("”",'''”,''' ,pag.get_text())
    spacy_doc= spa(pag)
    for sent in spacy_doc.sents :
        
        if len(sent.text) > 150 and text !="":
            sentances.append(sent)
            
        else :
            text= text+sent.text

        if len(text)>150:
            sentances.append(spa(text))
            text= ""
    
entities =[[entity.text for entity in sent.ents if entity.label_ not in['CARDINAL','DATE','TIME','QUANTITY','WORK_OF_ART']] for sent in sentances ]

In [71]:
len(spa("hello world").text)

11

In [452]:
df = pd.DataFrame(data={"sentence":sentances,'entities':entities})
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8271 entries, 0 to 8270
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  8271 non-null   object
 1   entities  8271 non-null   object
dtypes: object(2)
memory usage: 129.4+ KB
None


,sentence,entities
0,"(Book, One, :, A, Song, of, Ice, and, Fire, \n...","[George R.R. Martin, Gared]"
1,"(Version, History, :, \n \n, 2.0, -, Reedited,...","[3/5/02, Ser Waymar Royce]"
2,"(\n , Gared, did, not, rise, to, the, bait, .,...",[Royce]
3,"(“, What, proof, have, we, ?, ”, ,, \n , “, W...",[Gared]
4,"(He, wished, it, had, been, later, \n, rather,...",[Royce]
...,...,...
8266,"(The, cream, -, and, -, gold, dragon, was, suc...",[]
8267,"(When, it, saw, Jorah, ,, it, raised, its, hea...","[Jorah, Wordless]"
8268,"(Jhogo, was, the, \n, first, to, lay, his, ara...","[Jhogo, first]"
8269,"(And, after, them, came, her, handmaids, ,, an...","[Dothraki, Dany, Drogo]"


# analysis sentences to improve character output

In [98]:
df["sentence"].loc[6]


  “We have a long ride before us,”, Gared pointed out.“Eight days, maybe nine.And night is 
falling.”, 
 Ser Waymar Royce glanced at the sky with disinterest.

In [ ]:
from spacy import displacy
displacy.render(spa("“Fallen,” Will insisted. “There’s one woman up an ironwood, halfhid in the branches. A far-eyes.” He smiled thinly. “I took care she never saw me. When I got closer, I saw that she wasn’t moving neither.” Despite himself, he shivered.\n“You have a chill?”, Royce asked., “Some,” Will muttered.,“The wind, m’lord.” "),style='ent')
doc= '''“You have a chill?”, Royce asked.'''
displacy.render(spa(doc),style='ent')
#add comma after each ” 
#make each sentace more than 150 character
#not all entity are charcters in the story 

# organize the data

In [453]:
#drop rows has no entiteies
df.drop(index=df[df["entities"].str.len()==0].index,inplace=True)


In [454]:
#for making sure the person is real person in the novel we did scraping game of theron wiki and get all characters in the novel 
# we will  compare result of ner with scraping result 
df_char = pd.read_csv('characters.csv')

In [455]:
# the
# ser
#prince joffrey’s	
#somtime characters referd by first name only and sometime other refer by full name
def filter_character(row):
    ll=[]
    for el in row:
        if re.sub("’[a-zA-Z]","",el.split(" ")[0].lower()) not in ["ser","prince","lady","lord","king","house"] :
            if re.sub("’[a-zA-Z]","",el.split(" ")[0].lower())  in df_char.firstName.values:
                ll.append(re.sub("’[a-zA-Z]","",el.split(" ")[0].lower()))
        elif len(el.split(" "))>1:
            if re.sub("’[a-zA-Z]","",el.split(" ")[1].lower())  in df_char.firstName.values:
                ll.append(re.sub("’[a-zA-Z]","","".join(el.lower().split(" ")[1])))

    return ll



In [456]:
df['character']= df['entities'].apply(filter_character)

#drop rows has no entiteies
df.drop(index=df[df["character"].str.len()==0].index,inplace=True)

df

,sentence,entities,character
0,"(Book, One, :, A, Song, of, Ice, and, Fire, \n...","[George R.R. Martin, Gared]","[george, gared]"
1,"(Version, History, :, \n \n, 2.0, -, Reedited,...","[3/5/02, Ser Waymar Royce]",[waymar]
2,"(\n , Gared, did, not, rise, to, the, bait, .,...",[Royce],[royce]
3,"(“, What, proof, have, we, ?, ”, ,, \n , “, W...",[Gared],[gared]
4,"(He, wished, it, had, been, later, \n, rather,...",[Royce],[royce]
...,...,...,...
8262,"(No, ,, she, wanted, to, shout, to, him, ,, no...",[Daenerys Stormborn],[daenerys]
8267,"(When, it, saw, Jorah, ,, it, raised, its, hea...","[Jorah, Wordless]",[jorah]
8268,"(Jhogo, was, the, \n, first, to, lay, his, ara...","[Jhogo, first]","[jhogo, first]"
8269,"(And, after, them, came, her, handmaids, ,, an...","[Dothraki, Dany, Drogo]",[drogo]


In [457]:
relation_list=[]
for i in range(0,df.shape[0]):
    relation_list.extend(df["character"].iloc[i])
    

In [458]:
src_list=[]
des_list = []
for i in range(len(relation_list)):
    if(i%2 ==0):
        src_list.append(relation_list[i])
    else:
        des_list.append(relation_list[i])

In [459]:
if len(src_list) > len(des_list):
    src_list.pop()
elif len(src_list) < len(des_list):
    des_list.pop()

print(len(src_list))
print(len(des_list))

3797
3797


In [460]:
relation_df = pd.DataFrame({"src":src_list,"des":des_list})
print(relation_df.info())
relation_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3797 entries, 0 to 3796
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   src     3797 non-null   object
 1   des     3797 non-null   object
dtypes: object(2)
memory usage: 59.5+ KB
None


,src,des
0,george,gared
1,waymar,royce
2,gared,royce
3,gared,gared
4,gared,first
...,...,...
3792,mirri,mormont
3793,mirri,jorah
3794,jorah,daenerys
3795,jorah,jhogo


In [461]:
# drop self relationsthip : gared gared
print("count of self relationship : ",relation_df[relation_df["src"]==relation_df["des"]].shape[0])
relation_df.drop(relation_df[relation_df["src"]==relation_df["des"]].index,inplace=True)


count of self relationship :  612


In [463]:
# a :b = b:a
import numpy as np
# relation_df.apply(lambda row : np.sort(row.values))
# ddf = np.sort(relation_df.values,axis=1)
relation_df = pd.DataFrame( np.sort(relation_df.values,axis=1),columns=["src","des"])
relation_df

,src,des
0,gared,george
1,royce,waymar
2,gared,royce
3,first,gared
4,gared,waymar
...,...,...
3180,mirri,mormont
3181,jorah,mirri
3182,daenerys,jorah
3183,jhogo,jorah


In [464]:
# for use in groubing
relation_df['value']= 1
relation_df = relation_df.groupby(by=["src","des"],sort=False,as_index=False).sum().sort_values("value",ascending=False)

In [465]:
relation_df

,src,des,value
38,ned,robert,129
28,catelyn,ned,44
435,jon,sam,39
116,joffrey,sansa,36
79,jon,ned,31
...,...,...,...
662,mya,sky,1
665,lysa,ned,1
666,little,lysa,1
669,heward,ned,1


# graph analysis and visualization

In [466]:
relation_df[relation_df["src"]=="daenerys"]

,src,des,value
162,daenerys,jorah,4
57,daenerys,first,4
53,daenerys,illyrio,3
50,daenerys,viserys,3
161,daenerys,drogo,2
46,daenerys,rhaegar,2
1267,daenerys,mirri,2
872,daenerys,jhiqui,1
886,daenerys,robert,1
1020,daenerys,hodor,1


In [467]:
import networkx as nx
G = nx.from_pandas_edgelist(relation_df[:500],source="src",target="des",edge_attr="value")


In [468]:
plt.figure(figsize=(15,15))
pos = nx.kamada_kawai_layout(G)
nx.draw(G,with_labels=True,node_color="skyblue",edge_cmap=plt.cm.Blues,pos= pos)
plt.show()

In [450]:
from pyvis.network import Network
net = Network(notebook=True,width= "1920px",height="1024px",bgcolor='black',font_color="white",cdn_resources='remote')

edges_num =dict(G.degree)
nx.set_node_attributes(G,edges_num,"size")

net.from_nx(G)
net.show("got.html")

got.html


DegreeView({'ned': 59, 'robert': 34, 'jon': 47, 'sam': 8, 'joffrey': 17, 'sansa': 23, 'arya': 22, 'catelyn': 40, 'dothraki': 16, 'jorah': 15, 'mormont': 5, 'first': 41, 'winterfell': 27, 'grenn': 4, 'varys': 4, 'stark': 21, 'tyrion': 22, 'rodrik': 11, 'samwell': 3, 'benjen': 7, 'viserys': 10, 'drogo': 12, 'alliser': 5, 'bronn': 17, 'vardis': 4, 'jeyne': 5, 'syrio': 5, 'nymeria': 4, 'jaime': 15, 'cersei': 6, 'barristan': 8, 'lysa': 10, 'riverrun': 14, 'grand': 6, 'arryn': 4, 'kingsguard': 9, 'mirri': 7, 'illyrio': 6, 'second': 7, 'halder': 4, 'lyanna': 3, 'tywin': 20, 'lannister': 18, 'rickon': 8, 'walder': 5, 'eyrie': 11, 'casterly': 6, 'pycelle': 6, 'aerys': 3, 'stannis': 6, 'jhogo': 7, 'brandon': 6, 'yoren': 4, 'kevan': 2, 'vayon': 2, 'bowen': 2, 'rhaegar': 5, 'petyr': 5, 'ghost': 2, 'greyjoy': 2, 'theon': 7, 'hodor': 5, 'gregor': 12, 'renly': 3, 'loras': 5, 'haggo': 2, 'qotho': 6, 'irri': 8, 'jhiqui': 5, 'mya': 2, 'sandor': 5, 'tobho': 1, 'castle': 2, 'masha': 1, 'jory': 5, 'pentos'